## Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

In [2]:
#training data
#inputs -> temp, humidity, rainfall
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 76, 90]], dtype='float32')

#targets -> apples, oranges
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [3]:
#inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

#### Model

In [4]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.2903, -0.7678, -0.4408],
        [ 0.4838, -1.2735, -1.0965]], requires_grad=True)
tensor([-1.0387,  1.0605], requires_grad=True)


In [5]:
#model
def model(x):
    return x @ w.t() + b        #@ -> matrix multiplication in pytorch; t-> transpose

In [6]:
#predictions
preds = model(inputs)
print(preds)

tensor([[ -50.2407,  -96.0960],
        [ -70.3948, -137.1581],
        [-104.2298, -191.0963],
        [ -20.7500,  -44.9205],
        [ -79.0284, -161.0307]], grad_fn=<AddBackward0>)


In [7]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
#loss function (mse)
def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [9]:
loss = mse(preds, targets)
loss.sqrt()

tensor(198.3620, grad_fn=<SqrtBackward0>)

In [10]:
loss.backward()

In [11]:
print(w)
print(w.grad)

tensor([[ 0.2903, -0.7678, -0.4408],
        [ 0.4838, -1.2735, -1.0965]], requires_grad=True)
tensor([[-11574.7891, -13205.2139,  -9033.8496],
        [-17934.3594, -20064.0352, -13883.1318]])


In [12]:
#resetting gradients back to 0 to prevent adding up
w.grad.zero_()
b.grad.zero_()
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


##### Gradient Descent

In [13]:
preds = model(inputs)
preds

tensor([[ -50.2407,  -96.0960],
        [ -70.3948, -137.1581],
        [-104.2298, -191.0963],
        [ -20.7500,  -44.9205],
        [ -79.0284, -161.0307]], grad_fn=<AddBackward0>)

In [14]:
loss = mse(preds, targets)
loss

tensor(39347.4648, grad_fn=<DivBackward0>)

In [15]:
#compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-11574.7891, -13205.2139,  -9033.8496],
        [-17934.3594, -20064.0352, -13883.1318]])
tensor([-141.1287, -218.0603])


In [16]:
#adjust weights and reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [17]:
#new weights and biases
print(w)
print(b)

tensor([[ 0.4061, -0.6357, -0.3504],
        [ 0.6632, -1.0729, -0.9577]], requires_grad=True)
tensor([-1.0372,  1.0627], requires_grad=True)


In [18]:
#re-calculate loss
preds = model(inputs)
loss = mse(preds, targets)
loss

tensor(27429.9316, grad_fn=<DivBackward0>)

In [19]:
#training with 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [20]:
#new loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(806.7490, grad_fn=<DivBackward0>)


In [21]:
preds

tensor([[ 64.7374,  81.2204],
        [ 85.6087, 102.9022],
        [ 99.4771, 114.5799],
        [ 65.3718,  95.5143],
        [ 80.0208,  83.1270]], grad_fn=<AddBackward0>)

In [22]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

##### Linear regression using PyTorch built-ins

In [23]:
import torch.nn as nn

In [24]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

##### Tensor Dataset and Dataloader

In [25]:
from torch.utils.data import TensorDataset

In [26]:
#define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [27]:
from torch.utils.data import DataLoader

In [28]:
#define dataloader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)


In [29]:
i=1
for xb, yb in train_dl:
    print(f'batch {i}:')
    print(xb)
    print(yb)
    i+=1

batch 1:
tensor([[ 69.,  96.,  70.],
        [103.,  43.,  36.],
        [ 92.,  87.,  64.],
        [ 87., 134.,  58.],
        [ 88., 134.,  59.]])
tensor([[103., 119.],
        [ 20.,  38.],
        [ 82., 100.],
        [119., 133.],
        [118., 132.]])
batch 2:
tensor([[ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 73.,  66.,  44.],
        [ 87., 135.,  57.],
        [ 68.,  97.,  70.]])
tensor([[ 57.,  69.],
        [ 80., 102.],
        [ 57.,  69.],
        [118., 134.],
        [102., 120.]])
batch 3:
tensor([[ 91.,  88.,  64.],
        [101.,  44.,  37.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 68.,  96.,  71.]])
tensor([[ 81., 101.],
        [ 21.,  38.],
        [ 22.,  37.],
        [ 56.,  70.],
        [104., 118.]])


In [30]:
#do this instead of manually initializing weights and biases
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5012,  0.5430, -0.4458],
        [-0.4275,  0.5697, -0.0451]], requires_grad=True)
Parameter containing:
tensor([-0.4064,  0.3237], requires_grad=True)


In [31]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.5012,  0.5430, -0.4458],
         [-0.4275,  0.5697, -0.0451]], requires_grad=True),
 Parameter containing:
 tensor([-0.4064,  0.3237], requires_grad=True)]

In [32]:
preds = model(inputs)
preds

tensor([[-19.7842,   5.3406],
        [-26.7651,   8.6603],
        [  2.8933,  36.8464],
        [-44.6775, -20.4596],
        [-14.0687,  22.3529],
        [-20.8285,   4.3434],
        [-27.7539,   8.0455],
        [  1.9463,  36.3737],
        [-43.6332, -19.4624],
        [-14.0133,  22.7353],
        [-20.7731,   4.7258],
        [-27.8094,   7.6631],
        [  3.8822,  37.4612],
        [-44.7329, -20.8420],
        [-13.0245,  23.3502]], grad_fn=<AddmmBackward0>)

In [33]:
#loss function
import torch.nn.functional as F

In [34]:
loss_fn = F.mse_loss

In [35]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8356.1846, grad_fn=<MseLossBackward0>)


##### Optimizer using Stochastic Gradient Descent

In [37]:
#define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [38]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        #train with batches of data
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step() #update params using gradient
            opt.zero_grad()
        
        if (epoch+1)%10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [39]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 85.05235290527344
Epoch [20/100], Loss: 69.84447479248047
Epoch [30/100], Loss: 47.959476470947266
Epoch [40/100], Loss: 58.776336669921875
Epoch [50/100], Loss: 53.681358337402344
Epoch [60/100], Loss: 56.428184509277344
Epoch [70/100], Loss: 41.943817138671875
Epoch [80/100], Loss: 64.53816223144531
Epoch [90/100], Loss: 35.160160064697266
Epoch [100/100], Loss: 64.22805786132812


In [40]:
preds = model(inputs)
preds

tensor([[ 57.4244,  70.8166],
        [ 77.3265,  96.6175],
        [126.7653, 139.8927],
        [ 24.5404,  40.3683],
        [ 91.5214, 110.3018],
        [ 56.1241,  69.6785],
        [ 76.3451,  96.0287],
        [126.6142, 140.1208],
        [ 25.8407,  41.5064],
        [ 91.8402, 110.8511],
        [ 56.4430,  70.2278],
        [ 76.0262,  95.4795],
        [127.7467, 140.4814],
        [ 24.2216,  39.8190],
        [ 92.8216, 111.4399]], grad_fn=<AddmmBackward0>)

In [41]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])